# *Install Libraries* 

In [1]:
pip install -r ../config/requirements.txt

Note: you may need to restart the kernel to use updated packages.


# *Import Libraries*

In [2]:
import importlib.util
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import create_database, database_exists

spec = importlib.util.spec_from_file_location("model_structure", "../db_model/models_structure.py")
model_structure = importlib.util.module_from_spec(spec)
spec.loader.exec_module(model_structure)

# *Processing Data*

### *Reading* ###

In [3]:
candidates_dataframe = pd.read_csv("../data/candidates.csv", sep = ";")

In [4]:
candidates_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   First Name                 50000 non-null  object
 1   Last Name                  50000 non-null  object
 2   Email                      50000 non-null  object
 3   Application Date           50000 non-null  object
 4   Country                    50000 non-null  object
 5   YOE                        50000 non-null  int64 
 6   Seniority                  50000 non-null  object
 7   Technology                 50000 non-null  object
 8   Code Challenge Score       50000 non-null  int64 
 9   Technical Interview Score  50000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 3.8+ MB


*Seeing the names of the columns, a renaming will be done to have a better structure.*

*It can also be appreciated that there are no null data and that we work with objects and integers.*

In [5]:
columns_name = ['first_name', 'last_name', 'email', 'applicant_date', 'country', 
'experience_year', 'seniority', 'technology', 'code_challenge_score', 'technical_interview_score']
candidates_dataframe.columns = columns_name
candidates_dataframe.index += 1
candidates_dataframe.reset_index(inplace=True)
candidates_dataframe.rename(columns={'index': 'id'}, inplace=True)

### *Creating DataBase Connection* ###

In [6]:
with open('../config/credentials.json', 'r') as json_file:
    data = json.load(json_file)
    host = data["host"]
    database = data["database"]
    user = data["user"]
    password = data["password"]

database_connection = f"mysql://{user}:{password}@{host}/{database}"
engine = create_engine(database_connection)
if not database_exists(engine.url):
    create_database(engine.url)

model_structure.BASE.metadata.drop_all(engine)
model_structure.BASE.metadata.create_all(engine)

### *Upload Data to Data Base* ###

In [7]:
candidates_dataframe.to_sql("candidates", engine, if_exists='replace', index=False)

50000

### *Check Uploaded Data* ###

![Loaded Data](../notebooks/Loaded%20Data.png)

*Here, it can already be seen how the data has been loaded into the database and are ready to start with their subsequent analysis.*